### Loading TL;DR - Reddit Human Feedback Dataset
**M. Völske, M. Potthast, S. Syed, and B. Stein. Tl; dr: Mining reddit to learn automatic
summarization. In Proceedings of the Workshop on New Frontiers in Summarization, pages
59–63, 2017**

In [1]:
import json
import random
from datasets import load_dataset

dataset = load_dataset("CarperAI/openai_summarize_comparisons")
train_dataset = dataset["train"]
val_dataset = dataset["valid2"]

/opt/homebrew/Caskroom/miniconda/base/envs/rlhf-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 92534
})

In [3]:
# Labeling dataset to 2 candidates: [ candidate_0, candidate_1 ] and output: choice: 0 or 1
def label_data(sample):
  input_text = sample["prompt"] + " [summary]: "
  
  if random.random() < 0.5:
    candidate_0 = sample["chosen"]
    candidate_1 = sample["rejected"]
    choice = 0
  else:
    candidate_1 = sample["chosen"]
    candidate_0 = sample["rejected"]
    choice = 1
    
  return {
    "input_text": input_text,
    "candidate_0": candidate_0,
    "candidate_1": candidate_1,
    "choice": choice
  }
  
preprocessed_dataset = train_dataset.map(label_data, num_proc=4)
preprocessed_dataset.to_json("../data/large_preference.jsonl", orient="records", lines=True)

with open("../data/validate2_preference.jsonl", "w") as f:
  for sample in val_dataset:
    formatted = {
      'input_text': sample["prompt"] + " [summary]: ",
      'chosen': sample["chosen"],
      "rejectetd": sample["rejected"],
    }
    f.write(json.dumps(formatted) + "\n")

Creating json from Arrow format: 100%|██████████| 93/93 [00:01<00:00, 63.62ba/s]


In [4]:
preprocessed_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected', 'input_text', 'candidate_0', 'candidate_1', 'choice'],
    num_rows: 92534
})

In [5]:
val_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 50715
})